# Final Code

Here is the final code for image processing. I have compiled all my code into one file to make it easier to see.  

List of Files:

- binary.py

- bw_colormapping.py

- decode.py

- flipping.py

- hsv.py

- noiseColor.py

- occupancy.py

- purple_mapping.py

- restore_color.py

- rotate.py

- rotation.py

- test_gf.py

In [ ]:
# Encodes a binary message into image
from PIL import Image

message = '1001001 1101100 1101111 1110110 1100101 1101101 1101111 1101110 1100101 1111001'
message = message.replace(' ', '')
length = len(message)
counter = 0
image = Image.open("images/flipped.png")
data = image.load()
for y in range(image.height):
    for x in range(image.width):
        r,g,b,_ = data[x,y]
        mask  = 254
        r = r & mask
        g = g & mask
        b = b & mask
        if(counter < length):
            r = r | int(message[counter])
            counter+=1
        if(counter < length):
            g = g | int(message[counter])
            counter+=1
        if(counter < length):
            b = b | int(message[counter])
            counter+=1
        data[x,y] = r,g,b
image.save('images/binary-zebra.png')

In [ ]:
# Converts images to black and white
from PIL import Image

image = Image.open("images/godfather.png")
data = image.load()

for y in range(image.height):
    error = 0
    for x in range(image.width):
        pixel = data[x,y]
        k = (pixel[0]+pixel[1]+pixel[2])//3


        k+= error

        out = 0
        if k >= 128:
            out = 255
        
        data[x,y] = (out, out, out)

image.save("godf_bw_remap3.png")

In [ ]:
# Decodes the Binary file
from PIL import Image
message = []

image = Image.open('images/binary-zebra.png')
data = image.load()
for y in range(image.height):
    for x in range(image.width):
        r,g,b,_ = data[x,y]
        mask  = 1
        r = r & mask
        g = g & mask
        b = b & mask
        message.append(str(r))
        message.append(str(g))
        message.append(str(b))

check = ''.join(message)
counter = 0
for i in range(len(check)): 
    if i%7 == 0:
        if check[counter:i] != '0000000':
            print(check[counter:i], end=' ')
        counter = i

In [ ]:
# Flips images
from PIL import Image

image = Image.open("image.jpeg")

data = image.load()
print(image.width)
print(image.height)

def flip_horizontal(image):
    blank = Image.new("RGB",(image.width, image.height))
    bl_data = blank.load()

    for y in range(image.height):
        for x in range(image.width):
            pixel = data[x,y]
            bl_data[(image.width-1)-x,y] = pixel
    return blank

def flip_vertical(image):
    blank = Image.new("RGB",(image.width, image.height))
    bl_data = blank.load()

    for y in range(image.height):
        for x in range(image.width):
            pixel = data[x,y]
            bl_data[x,image.height-1-y] = pixel
    return blank

flipped_image = flip_vertical(image)
flipped_image.save("flipped_image.jpeg")


In [ ]:
# Contains functions for HSV Color Code
def rgb_to_hsv(r,g,b): 
    _max = max(r,g,b)
    _min = min(r,g,b)
    _diff = _max-_min

    #black exception
    if _max == 0:
        return(0,0,0)
    
    v_255 = _max
    s_1 = _diff/_max

    if _diff == 0:
        hue_360 = 0
    else: 
        if r == _max:
            hue_360 = 0 + 60*(g-b)/_diff

        elif g == _max:
            hue_360 = 120 + 60*(b-r)/_diff
        
        else: #b == _max:
            hue_360 = 240 + 60*(r-g)/_diff

    return (hue_360/360, s_1, v_255/255)

def hsv_to_rgb(h,s,v):
    _max = v
    _min = v - s*v
    _diff = (_max - _min)

    base = 0

    if 1/6<h<=3/6:
        base = 120
    if 3/6<h<=5/6:
        base = 240

    rotate_pos = True
    if 0<=h<1/6 or 2/6<=h<3/6 or 4/6<=h<5/6:
        rotate_pos = False
    if rotate_pos:
        _mid = (h*360-base) * _diff/60+_min
    else:
        _mid = (base-360*h) * _diff/60+_min

    if base == 0:
        if rotate_pos:
            return (_max, _mid, _min)
        else:
            return(_max, _mid, _min)



print(rgb_to_hsv(235, 64, 52))

In [ ]:
# Isolates the main focus in the image
from PIL import Image

im = Image.open('images/image.jpeg').convert("RGB")
pixels = im.load()
new_image = Image.new("RGB", (im.size))
new_data = new_image.load()

for y in range(im.height):
    for x in range(im.width):
        pixel = pixels[x,y]

        r=pixel[0]
        g=pixel[1]
        b=pixel[2]

        if(g > 200 or b > 100 or r > 130 ):
            new_data[x,y] = (r,g,b)
        else: 
            new_data[x,y] = (0,0,0)   #black pixel channel

new_image.save("images/isolated.jpeg")

conversion = Image.open("images/isolated.jpeg")

In [ ]:
# Creates noise of an image
from PIL import Image
from random import randint as rand
def add_color_tint(image_path):
    # Open the grayscale image
    img = Image.open(image_path)
    data = img.load()
    
    # Create a new RGB image
    color_img = Image.new("RGB", img.size)
    color_load = color_img.load()

    for x in range(img.width):
        for y in range(img.height):
            pixel = data[x, y]
            r = rand(pixel[0], 255)
            g = rand(pixel[1], 255)
            b = rand(pixel[2], 255)
            color_load[x, y] = (r, g, b)

    # Save the color-restored image
    color_img.save("./images/restored_frog45.png")

# Main code
add_color_tint("./images/zebra.png")

In [ ]:
# This creates an image overlay
from PIL import Image

img1 = Image.open('images/flipped2.png')
data1 = img1.load()

img2 = Image.open('images/Zorboi.png')
data2 = img2.load()

print("Height:\t",img1.height, img2.height)
print("Width:\t", img1.width, img2.width)

new_image = Image.new("RGB", (img2.size))
new_data = new_image.load()
# Adjust the occupancy to configure brightness.
ocp = .75 
ocp2 = .75

for x in range(img1.width):
    for y in range(img1.height):
        r1,g1,b1= data1[x, y]
        r1 = int(r1 * ocp)
        g1 = int(g1 * ocp)
        b1 = int(b1 * ocp)

        r2,g2,b2,_= data2[x,y]

        r2 = int(r2 * ocp2)
        g2 = int(g2 * ocp2)
        b2 = int(b2 * ocp2)

        nr = int(r2 + r1 // 2)
        ng = int(g2 + g1 // 2)
        nb = int(b2 + b1 // 2)

        new_data[x, y] = (nr,ng,nb)

new_image.save('transp3.png')

In [ ]:
# Draws purple over the green pixels
from PIL import Image

im = Image.open('images/frog.png').convert("RGB")
pixels = im.load()
p = (128, 0, 128)

for x in range(im.width):
    for y in range(im.height):
        r, g, b = pixels[x, y]
        if g > 125:
            pixels[x, y] = p
            
im.save('images/frog_purp.png')

In [ ]:
# Adds color to an image
from PIL import Image
from random import randint as ran
#Convert to HSL formula from: https://www.rapidtables.com/convert/color/rgb-to-hsl.html
'''
Lum => to max - min of RGB /2 
In Grayscale it equals itself.

Plan to convert:
-Group luminace by pixel cords.
- Add a random color to place them into.
'''

im = Image.open('images/godfather.png').convert("RGB")
pixels = im.load()
colorDB = {}
lum = 0
new_image = Image.new("RGB", (im.size))
new_data = new_image.load()

for x in range(im.width):
    for y in range(im.height):
        r, g, b = pixels[x, y]
        lum = int(r/2.55)
        if lum not in colorDB:
            colorDB.setdefault(lum, [[x,y]])
        else:
            colorDB[lum].append([x,y])

'''
lum => 0-10 = Black(0,0,0)
lum(90, 100) = White(255,255,255)
lum(11,36) = Priority Blue
lum(37,62) = Priority Green
lum(63, 89) = Priority Red
'''



for lum in colorDB:
    if 0 <= lum <= 10:
        color = (0, 0, 0)
    elif 90 <= lum <= 100:
        color = (255, 255, 255)
    elif 63 <= lum <= 89: 
        color = (int(ran(0,255)*0.314),int(ran(0,255)*0.785),int(ran(0,255)*0.499))
    elif 37 <= lum <= 62: 
        color = (int(ran(0,255)*0.785),int(ran(0,255)*0.499),int(ran(0,255)*0.314))
    elif 11 <= lum <= 36: 
        color = (int(ran(0,255)*0.499),int(ran(0,255)*0.314),int(ran(0,255)*0.785))
    else:
        color = (0, 0, 0)
    # print(colorDB[key])
    for i in colorDB[lum]:
        x = i[0]
        y = i[1]
        new_data[x,y] = color

# for x in range(im.width):
#     for y in range(im.height):
#         r, g, b = pixels[x, y]
#         new_data[x,y] = colorDB[r].pop()

new_image.save('check_color7.png')

In [ ]:
# Rotate image
from PIL import Image
import math
image = Image.open('images/zebra.png')
data = image.load()
new_image = Image.new("RGB", (image.size))
new_data = new_image.load()

rotation = 0.5
center = (image.width/2, image.height/2)

cos_thta = math.cos(-rotation)
sine_thta = math.sin(-rotation)
dx = - cos_thta * image.width/2 + sine_thta*image.height/2+image.width/2
dy = - sine_thta * image.width/2 - cos_thta*image.height/2+image.heigh/2

#dx = 0
#dy = 0



for y in range(image.height):
    for x in range(image.width):

        new_x = cos_thta * x - sine_thta * y + dx
        new_y = sine_thta * x + cos_thta * y + dy
        new_x //= 1
        new_y //= 1

        if 0<=new_x<image.width and 0<=new_y<image.height:
            new_data[x,y] = data[new_x,new_y]
        else:
            new_data[x,y] = (0,0,0)


new_image.save('check2.png')

In [ ]:
# rotate image
from PIL import Image
import math
image = Image.open('images/zebra.png')
data = image.load()
new_image = Image.new("RGB", (image.size))
new_data = new_image.load()

rotation = 0.5
center = (image.width/2, image.height/2)
for y in range(image.height):
    for x in range(image.width):
        dx = x - center[0]
        dy = y - center[1]

        original_radius = math.sqrt(dx**2+dy**2)
        original_angle=math.atan2(dy,dx)
        new_angle = original_angle - rotation
        sample_x = math.cos(new_angle) * original_radius + center[0]
        sample_y = math.sin(new_angle) * original_radius + center[1]
        sample_x //=1
        sample_y//=1

        if 0 <= sample_x < image.width and 0 <= sample_y < image.height:
            new_data[x,y] = data[sample_x,sample_y]
        else:
            new_data[x,y] = (0,0,0)

new_image.save('check.png')

In [ ]:
# Adds grid based coloring with greyscale
from PIL import Image

image = Image.open("images/godfather.png")
data = image.load()

error_matrix = []
for y in range(image.width+1):
    row = []
    for x in range(image.height+1):
        row.append(0)
    error_matrix.append(row)

for y in range(image.height):
    error = 0
    for x in range(image.width):
        pixel = data[x,y]
        k = (pixel[0]+pixel[1]+pixel[2])//3
        # data[x,y] = (k, k, k)

        # k+= error
        k += error_matrix[x][y]

        out = 0
        if k >= 128:
            out = 255
        
        data[x,y] = (out, out, out)

        error = k - out
        quarter_error = error/4

        error_matrix[x+1][y] += quarter_error
        error_matrix[x-1][y+1] += quarter_error
        error_matrix[x][y+1] += quarter_error
        error_matrix[x+1][y+1] += quarter_error


image.save("godf_bw_remap5.png")